# What apps are most liked by users

- This goal of this project is to analyze what type of apps attract more users.

We will use two datasets.
1.[Google play app data](https://www.kaggle.com/lava18/google-play-store-apps) 
2.[Apple Mobile App Data](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps)

In [1]:
from csv import reader

def explore_data(dataset,start,end,rows_and_columns=False):
    dataset_slice = dataset[start:end]
    for row in dataset_slice:
        print(row)
        print('\n')
        
    if rows_and_columns:
        print('Number of rows:',len(dataset))
        print('Number of columns:', len(dataset[0]))
        print('\n')
    
        
def open_data(file_name):
    file = open(file_name,encoding='utf8')
    read_file = reader(file)
    dataset = list(read_file)
    return dataset
    
ios_file=open_data('AppleStore.csv')
google_file=open_data('googleplaystore.csv')

In [2]:
#explore_data(ios_file[1:],1,2,True)
explore_data(google_file[1:],1,2,True)

['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


Number of rows: 10841
Number of columns: 13




In [3]:
print(google_file[0:2])

[['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver'], ['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']]


In [4]:
print(google_file[10473])
print(len(google_file[10473]))

['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']
12


In [5]:
del google_file[10473]
print(google_file[10473])
print(len(google_file[10473]))


['osmino Wi-Fi: free WiFi', 'TOOLS', '4.2', '134203', '4.1M', '10,000,000+', 'Free', '0', 'Everyone', 'Tools', 'August 7, 2018', '6.06.14', '4.4 and up']
13


# found duplicate data from disscussion 
From the [disscussion](https://www.kaggle.com/lava18/google-play-store-apps/discussion) we can see that there may be duplicate datas. 
So I am going to make a function which finds out duplicate data entries. 

1.First I am going to find what kind of data has duplicate entries
- i will try to find apps with dupliactes entries

In [6]:
print(google_file[0])

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


In [7]:
def find_duplicates(dataset):
    duplicate = []
    unique_app = []
    for row in dataset[1:]:
        if row[0] in unique_app:
            duplicate.append(row[0])
        else:
            unique_app.append(row[0])
    
    print('Number of duplicate app: ',len(duplicate))
    print('Example of duplicate app:', duplicate[:15])
    return unique_app,duplicate
            

Here we will first get all the duplicates app by the `find_duplicates` function
the function return alist the name of the app that has duplicate entries.

-> `find dupliactes` has two parameters. dataset and header 
-> if your dataset doesnt have header row, pass `header=False` as parameter. 
->The default value is `True` as we assume that the dataset has header while being passed as arguement 

In [8]:
android_unique_apps , android_duplicate = find_duplicates(google_file)

Number of duplicate app:  1181
Example of duplicate app: ['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings', 'Box', 'Zenefits', 'Google Ads', 'Google My Business', 'Slack', 'FreshBooks Classic', 'Insightly CRM', 'QuickBooks Accounting: Invoicing & Expenses', 'HipChat - Chat Built for Teams', 'Xero Accounting Software']


-> the `find dupliactes` return two values. `unique apps` and apps that have `duplicate values` in the dataset

# Cleaning out duplicates

- listing the unique apps with their maximum ratings


- we can see here that the rating are in 3rd colomn
- ['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']

- We have a list `android_unique_apps` containg the name of unique app . we will use this same technique to extract out the unique apps' maximum rating.



In [9]:
unique_app_max_rating={}
for row in google_file[1:]:
    name = row[0]
    rating = float(row[3])
    if name in unique_app_max_rating:
        unique_app_max_rating[name]=max(unique_app_max_rating[name],rating)
    else:
        unique_app_max_rating[name]=rating
                                                                        

We have seen there is 1181 duplicate entries in the dataset. 

In [10]:
print(len(google_file[1:]))

10840


We can see that after deleting the duplicate entries , we have exact `9659` values. 

In [11]:
print(unique_app_max_rating['Instagram'])

66577446.0


- `we'd like to analyze only the apps that are directed toward an English-speaking audience. However, if we explore the data long enough, we'll find that both data sets have apps with names that suggest they are not directed toward an English-speaking audience.`


- At first we counted the unique apps with their max ratings.

- Then we looped throug the data and check if the row contains the maximum rating 

- we will also keep count of the already visited apps so that duplicate entries with max ratings also get eliminated

In [12]:
android_header = google_file[0]
android_clean = []
already_added = []
for row in google_file[1:]:
    name=row[0]
    rating=float(row[3])
    max_rating=unique_app_max_rating[name]
    if (rating==max_rating) and (name not in already_added):
        #(row)
        android_clean.append(row)
        already_added.append(name)
        
#print('sesh')
print(len(android_clean))

9659


`android_clean` is the clean data without duplicates.

# Remiving NON English App

We're not interested in keeping these apps, so we'll remove them. One way to go about this is to remove each app with a name containing a symbol that is not commonly used in English text — English text usually includes letters from the English alphabet, numbers composed of digits from 0 to 9, punctuation marks (., !, ?, ;), and other symbols (+, *, /).

Behind the scenes, each character we use in a string has a corresponding number associated with it. For instance, the corresponding number for character 'a' is 97, character 'A' is 65, and character '爱' is 29,233. We can get the corresponding number of each character using the ord() built-in function.

In [13]:
print(ord('a'))
print(ord('z'))
print(ord('A'))
print(ord('Z'))

97
122
65
90


- part 1
  - we will keep names with ascii values
     - we have 128 ascii characters indexed from 0 to 127

In [14]:
def is_english(name):
    for letter in name:
        nvalue = ord(letter)
        if nvalue> 127:
            return False
    return True           

This function works just fine. But its wont work with all the english names bacause some english apps may use special characters such as
- `Docs To Go™ Free Office Suite` returns it as a non english name.

In [15]:
print(is_english('Docs To Go™ Free Office Suite'))
print(is_english('Instachat 😜'))

False
False


- part 2
    - If we're going to use the function we've created, we'll lose useful data since many English apps will be incorrectly labeled as non-English. To minimize the impact of data loss, we'll only remove an app if its name has more than three characters with corresponding numbers falling outside the ASCII range. This means all English apps with up to three emoji or other special characters will still be labeled as English. Our filter function is still not perfect, but it should be fairly effective.

In [16]:
def is_english(name):
    cnt = 0
    for letter in name:
        nvalue = ord(letter)
        if nvalue> 127:
            cnt +=1
    if cnt>3:
        return False
    return True   

print(is_english('Docs To Go™ Free Office Suite'))
print(is_english('Instachat 😜'))

True
True


# removing non english app from android

In [17]:
print(android_header)

android_english=[]
for row in android_clean:
    name=row[0]
    if is_english(name)==True:
        android_english.append(row)


['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


In [18]:
explore_data(android_english,1,3,True)

['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


Number of rows: 9614
Number of columns: 13




# removing non english apps from ios 

In [22]:
print(len(ios_file))
ios_header = ios_file[0]
ios_data = ios_file[1:]
print(ios_header)

ios_english=[]
for row in ios_data:
    name=row[1]
    if is_english(name)==True:
        ios_english.append(row)
        
explore_data(ios_english,1,3,True)

7198
['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']
['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


Number of rows: 6183
Number of columns: 16


